# SPR 2026 - FastText + Logistic Regression

**FastText vs Word2Vec**

FastText usa subword embeddings, lidando melhor com:
- Palavras OOV (out-of-vocabulary)
- Variações morfológicas
- Termos médicos com prefixos/sufixos comuns

---
**CONFIGURAÇÃO OFFLINE:**
1. No Kaggle, vá em Settings → Internet → **OFF**
2. Gensim já está pré-instalado no Kaggle
---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from gensim.models import FastText
import re
import warnings
warnings.filterwarnings('ignore')

SEED = 42
EMBEDDING_DIM = 100
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
np.random.seed(SEED)
print('Bibliotecas carregadas!')

In [ ]:
# Carregar dados
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

In [ ]:
# Preprocessamento
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', ' ', text)
    return text.split()

train['tokens'] = train['report'].apply(preprocess)
test['tokens'] = test['report'].apply(preprocess)

print(f'Exemplo: {train["tokens"].iloc[0][:10]}')

In [ ]:
# Treinar FastText (usa n-grams de caracteres)
all_texts = train['tokens'].tolist() + test['tokens'].tolist()

ft = FastText(
    sentences=all_texts,
    vector_size=EMBEDDING_DIM,
    window=5,
    min_count=2,
    min_n=3,  # min n-gram length
    max_n=6,  # max n-gram length
    workers=4,
    epochs=10,
    seed=SEED
)

print(f'Vocabulário: {len(ft.wv)} palavras')
print('FastText pode gerar embeddings para palavras OOV!')

In [ ]:
# Converter textos em embeddings (Mean Pooling)
# FastText pode gerar embedding para qualquer palavra (mesmo OOV)
def text_to_embedding(tokens, model, dim):
    if len(tokens) == 0:
        return np.zeros(dim)
    vectors = [model.wv[w] for w in tokens]  # FastText sempre retorna embedding
    return np.mean(vectors, axis=0)

X_train = np.array([text_to_embedding(t, ft, EMBEDDING_DIM) for t in train['tokens']])
X_test = np.array([text_to_embedding(t, ft, EMBEDDING_DIM) for t in test['tokens']])
y_train = train['target'].values

print(f'X_train shape: {X_train.shape}')

In [ ]:
# Treinar Logistic Regression
model = LogisticRegression(
    C=1.0,
    max_iter=1000,
    class_weight='balanced',
    random_state=SEED,
    n_jobs=-1
)

model.fit(X_train, y_train)
print('Modelo treinado!')

In [ ]:
# Predições e submissão
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print('submission.csv criado!')
print(submission['target'].value_counts().sort_index())